In [6]:
import cv2
import numpy as np
import scipy.linalg as la

In [17]:
samples_per_second = 5
stride = 20

In [18]:
def frame_distance(frame1, frame2, stride = 20):

    frame1_strided = frame1[::stride, ::stride, :].reshape(-1 , 3).astype(int)
    frame2_strided = frame2[::stride, ::stride, :].reshape(-1 , 3).astype(int)

    x = la.norm(
        frame1_strided - frame2_strided,
        ord=2,
        axis=1
        ).mean()

    # Normalize for number of pixels
    return x

In [19]:
# Start the video capture
vc = cv2.VideoCapture("video/2021-09-16_16-16-44.mp4")

# Get the frames per second and total frames
fps = vc.get(cv2.CAP_PROP_FPS)
total_frames = vc.get(cv2.CAP_PROP_FRAME_COUNT)
print(f"Total frames: {total_frames}")

skip_frames = fps / samples_per_second
frame_comps = int(total_frames // skip_frames)
print(f"Frame comparisons: {frame_comps}")

Total frames: 354.0
Frame comparisons: 29


In [20]:
curr_frame = None
prev_frame = None
frame_diffs = np.zeros(frame_comps)
frame_pos = 0
idx = 0

while vc.isOpened():

    ret, frame = vc.read()
    frame_pos += skip_frames
    vc.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)

    if ret:

        if curr_frame is None:
            curr_frame = frame
        else:
            prev_frame = curr_frame
            curr_frame = frame

            frame_diffs[idx] = frame_distance(curr_frame, prev_frame, stride=stride)
            idx += 1

    else:
        vc.release()
        break


In [21]:
frame_diffs

array([20.30465017, 20.64087169, 17.05901155, 17.11878974, 17.25949156,
       16.74637562, 14.63633698, 13.8993657 , 17.36125051, 15.25941241,
       19.61621908, 26.7124042 , 16.39826633, 19.78830752, 29.79474898,
       35.18015411, 33.63303351, 32.42449592, 31.78823633, 29.32355771,
       32.44528329, 23.71549195, 21.75593342, 21.20205608, 24.01542527,
       29.38418172, 24.42146724, 66.73391644,  3.63544002])